Based on this article: https://towardsdatascience.com/u-net-deep-learning-colourisation-of-greyscale-images-ee6c1c61aabe

All training was carried out on a Nvidia Tesla K80 GPU with 12GB RAM and in less than 12 hours from start to finish with the progressive resizing.

Train size: 800 * sizes ~ 3200

dataset: https://data.vision.ee.ethz.ch/cvl/DIV2K/

In [1]:
from fastai.vision.models import *
from fastai.vision.models import resnet34
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile
from PIL import Image
from skimage import color
from datasets import load_dataset
from dotenv import load_dotenv
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from fastai.vision.all import *
from dataset import *
from model import *
import warnings
warnings.filterwarnings("ignore")

/home/prio/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))  # 0 corresponds to the first GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Working on device: ", device)

TRAIN_SIZE = 10
TEST_SIZE = 10

train_dataset, test_dataset = prepare_dataset(TRAIN_SIZE,TEST_SIZE,batch_size=8)
train_loader, test_loader = prepare_dataloader(train_dataset, test_dataset)



True
NVIDIA GeForce RTX 3050 Ti Laptop GPU
Working on device:  cuda:0
Dataset loaded successfully
Data loader prepared successfully


In [7]:
data = DataLoaders(train_loader, train_loader)

m = resnet34()
m = torch.nn.Sequential(*list(m.children())[:-2])
tst = DynamicUnet(m, 5, (256,256), norm_type=None)
learner = Learner(data,tst,loss_func=MSELossFlat, opt_func=Adam, metrics=[accuracy])
learner.fit_one_cycle(1, 1e-3)
x = cast(torch.randn(2, 3, 128, 128), TensorImage)
y = tst(x)
test_eq(y.shape, [2, 5, 128, 128])

epoch,train_loss,valid_loss,accuracy,time


KeyError: slice(None, 1, None)

In [ ]:
tst = DynamicUnet(m, 5, (128,128), norm_type=None)
x = torch.randn(2, 3, 127, 128)
y = tst(x)